In [25]:
import boto3
import os
import pandas as pd
import re
import warnings

warnings.filterwarnings(action='once')

In [26]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

s3_client = boto3.client(
    "s3",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    verify=False
)


### List Bucket Content

In [27]:
def s3_get_bucket_content(bucket, prefix="", print_content=True):

    content = list()

    response = s3_client.list_objects_v2(
        Bucket=bucket, Prefix=prefix)

    for obj in response.get("Contents", []):
        key = obj["Key"]
        if print_content:
            print(key)
        content.append(obj)

    return content

In [28]:
content = s3_get_bucket_content(bucket_name, prefix='/data//2023010')
content = s3_get_bucket_content(bucket_name, prefix='/fpg_test_2')

/data//20230101.csv
/data//20230102.csv
/data//20230103.csv
/data//20230104.csv
/data//20230105.csv
/data//20230106.csv
/data//20230107.csv
/data//20230108.csv
/data//20230109.csv
/fpg_test_2//sample1.csv
/fpg_test_2/sample1.csv


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3-openshift-storage.apps.mlops.software.bl.platform'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### Upload file

In [14]:
def s3_upload_file(bucket, local_folder, file_name, s3_folder):

    full_local_file_name = (local_folder + "/" + file_name).replace("//", "/")
    full_s3_file_name = (s3_folder + "/" + file_name).replace("//", "/")

    print(
        f"Copying: {full_local_file_name} \
        into S3: {full_s3_file_name}")

    s3_client.upload_file(
        full_local_file_name, bucket, Key=full_s3_file_name)


In [17]:
local_folder = '.'
file_name = 'sample1.csv'
s3_folder = '/fpg_test_2/'


s3_upload_file(bucket_name, local_folder, file_name, s3_folder)

Copying: ./sample1.csv         into S3: /fpg_test_2/sample1.csv


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3-openshift-storage.apps.mlops.software.bl.platform'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### Upload Folder

In [19]:
def s3_upload_folder(bucket, local_folder, s3_folder, mask=r".*"):

    for file in os.listdir(local_folder):
        if re.match(mask, file):
            s3_upload_file(bucket, local_folder, file, s3_folder)

    return

In [24]:
s3_upload_folder(
    bucket_name,
    './AI26X/pipelines/elyra-pipeline/resources/data/',
    '/fpg_test_4/',
    mask=r'.*.csv$')

Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230221.csv         into S3: /fpg_test_4/20230221.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230509.csv         into S3: /fpg_test_4/20230509.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20240315.csv         into S3: /fpg_test_4/20240315.csv


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3-openshift-storage.apps.mlops.software.bl.platform'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20240125.csv         into S3: /fpg_test_4/20240125.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230624.csv         into S3: /fpg_test_4/20230624.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230412.csv         into S3: /fpg_test_4/20230412.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20240205.csv         into S3: /fpg_test_4/20240205.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20240103.csv         into S3: /fpg_test_4/20240103.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230611.csv         into S3: /fpg_test_4/20230611.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230514.csv         into S3: /fpg_test_4/20230514.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20240229.csv         into S3: /fpg_test_4/20240229.csv
Copying: ./AI26X/pipelines/elyra-pipeline/resources/data/20230430.csv         into S3: /fpg_test_4/20230

### Download Folder